In [31]:
from sklearn import datasets
import pandas as pd
import csv

In [57]:
with open('IRIS.csv', 'rb') as csvfile:
    lines=csv.reader(csvfile)
# rb-> reading binary

In [43]:
"""
a=datasets.load_iris()
b=csv.reader(a)
data=list(a)
"""

'\na=datasets.load_iris()\nb=csv.reader(a)\ndata=list(a)\n'

In [38]:
#data

['data',
 'target',
 'frame',
 'target_names',
 'DESCR',
 'feature_names',
 'filename']

In [ ]:
# Iris dataset has 4 attributes-> sepal-width, sepal-length, petal-width, petal-length
# And we have to predict in which class these plant belong
# There are three classes-> Iris-santosa, iris-virginia, iris-versicolor

In [58]:
#opening our Iris csv file
with open('IRIS.csv') as csvfile:
    lines=csv.reader(csvfile)
    for row in lines:
        print(', '.join(row))    # joining all data in a string separating each data by ,

sepal_length, sepal_width, petal_length, petal_width, species
5.1, 3.5, 1.4, 0.2, Iris-setosa
4.9, 3, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.3, 0.2, Iris-setosa
4.6, 3.1, 1.5, 0.2, Iris-setosa
5, 3.6, 1.4, 0.2, Iris-setosa
5.4, 3.9, 1.7, 0.4, Iris-setosa
4.6, 3.4, 1.4, 0.3, Iris-setosa
5, 3.4, 1.5, 0.2, Iris-setosa
4.4, 2.9, 1.4, 0.2, Iris-setosa
4.9, 3.1, 1.5, 0.1, Iris-setosa
5.4, 3.7, 1.5, 0.2, Iris-setosa
4.8, 3.4, 1.6, 0.2, Iris-setosa
4.8, 3, 1.4, 0.1, Iris-setosa
4.3, 3, 1.1, 0.1, Iris-setosa
5.8, 4, 1.2, 0.2, Iris-setosa
5.7, 4.4, 1.5, 0.4, Iris-setosa
5.4, 3.9, 1.3, 0.4, Iris-setosa
5.1, 3.5, 1.4, 0.3, Iris-setosa
5.7, 3.8, 1.7, 0.3, Iris-setosa
5.1, 3.8, 1.5, 0.3, Iris-setosa
5.4, 3.4, 1.7, 0.2, Iris-setosa
5.1, 3.7, 1.5, 0.4, Iris-setosa
4.6, 3.6, 1, 0.2, Iris-setosa
5.1, 3.3, 1.7, 0.5, Iris-setosa
4.8, 3.4, 1.9, 0.2, Iris-setosa
5, 3, 1.6, 0.2, Iris-setosa
5, 3.4, 1.6, 0.4, Iris-setosa
5.2, 3.5, 1.5, 0.2, Iris-setosa
5.2, 3.4, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.6, 0.2, Iris-setos

In [82]:
type(lines)

_csv.reader

In [97]:
# now splitting the dataset in into training dataset that knn can use to amke predictions and
# a testing dataset to test the accuracy

# So first we need to convert the flower measure that will be loaded as string into numbers that we can work
# Next we need to split dataset randomly into train and test. At ratio of 67:33 for test:train is standard ratio for this purpose

import random
def loadDataset(filename, split, trainingSet=[], testSet=[]):
    with open(filename,'r') as csvfile:
        lines=csv.reader(csvfile)
        dataset=list(lines)
        for x in range(1,len(dataset)):
            for y in range(4):     #since there re 4 columns in iris
                dataset[x][y]=float(dataset[x][y])    # converting string values in to numbers
                
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])

In [98]:
trainingSet=[]
testSet=[]
loadDataset(r'IRIS.csv', 0.66, trainingSet, testSet)
print('Train: '+repr(len(trainingSet)))
print('Test: '+repr(len(testSet)))

Train: 94
Test: 56


In [219]:
# Now in order to make prediction we need to calculate the similarity between any two given instances
# this is needed so that we locate the K msot similar data instances in the training dataset and in turn make predictions
# Given that all four flower measurement are numeric and have same unit we can directly use Euclidean Distance
# Our approach will be to limit the Euclidean distance to a fixed length
import math
def euclideanDistance(instance1, instance2, length):   # inst1 and inst2 are the points between which we have to find the euclidean distance and length denotes how many attributes we want to include
    distance=0
    for x in range(length):
        distance+= pow(instance1[x]-instance2[x], 2)
    return math.sqrt(distance)    

In [220]:
data1=[2,2,2, 'a'] # belonging to class a
data2=[4,4,4, 'b'] # class b
distance=euclideanDistance(data1, data2, 3) # consider only first 3 elements
print('Distance:'+repr(distance))

Distance:3.4641016151377544


In [221]:
# now we have to look for K nearest neighbours
# now that we have a similarity measure we can use that to collect k most similar instances for a given unseen isntance
# well this is a strainght forward process of calculating the distance for all the isntances and selecting the subset with the smallest distance value

In [222]:
# For gettiing the smallest distance values; for that we have to be defining the fucntions getNeighbours
# it will return the k most similar neighbpours from the training set for given test instance
# From this fucntion will get k different point having least Euclidean distance from the test instance
import operator
def getNeighbours(trainingSet, testInstance, k):   # k is the number of nearest neighbours u want to check for
    distances=[]
    length=len(testInstance)-1
    for x in range(len(trainingSet)):
        dist=euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbours=[]
    for x in range(k):
        neighbours.append(distances[x][0])
    return neighbours

In [223]:
trainSet=[[2,2,2,'a'], [4,4,4, 'b']]
testSet=[5,5,5]
k=1
neighbours=getNeighbours(trainSet, testSet,k)
print(neighbours)

[[4, 4, 4, 'b']]


In [232]:
# Now we have to calculate the accuracy of our model
# An easy way to do taht is to get the ratio of correct prediction/total predictions
def getAccuracy(testSet, predictions):
    correct=0
    for x in range(len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return (correct/float(len(testSet)))*100.0

In [233]:
testSet=[[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
predictions=['a','a','a']
pred= getAccuracy(testSet,predictions)
print(pred)

66.66666666666666


In [235]:
def main():
    # prepare data
    trainingSet=[]
    testSet=[]
    split=0.67
    loadDataset(r'IRIS.csv', split, trainingSet, testSet)
    print('TrainSet: '+ repr(len(trainingSet)))
    print('TestSet: '+ repr(len(testSet)))
    
    #generate predictions
    predictions=[]
    k=3
    for x in range(len(testSet)):
        neighbors=getNeighbours(trainingSet, testSet[x], k)
        result=getResponse(neighbors)
        predictions.append(result)
        print('> predicted=' +repr(result) + 'actual=' + repr(testSet[x][-1]))
    accuracy=getAccuracy(testSet,predictions)
    print('Accuracy: '+ repr(accuracy)+'% ')
    
main()

TrainSet: 91
TestSet: 59
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-setosa'actual='Iris-setosa'
> predicted='Iris-versicolor'actual='Iris-versicolor'
> predicted='Iris-versicolor'actual='Iris-versicolor'
> predicted='Iris-versicolor'actual='Iris-versicolor'
> predicted='Iris-versicolor'ac